In [1]:
import torch
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import transformers
import numpy as np
from os import path
import pandas as pd
from transformers import AutoTokenizer, TFXLNetModel, TFAutoModel, TFXLNetForSequenceClassification
from constants import *
import dataset_utils as du
from new_model import create_model

In [2]:
# Hyperparameters
TRAIN_RATIO = .6
DEV_RATIO = .2
LEARNING_RATE = 0.000001 
BATCH_SIZE = 3
EPOCHS = 2

In [3]:
# Uncomment the following line to rerun pre-processing and tokenization
#prepare_and_save(RAW_DATASET_PATH, path.join(PROCESSED_DIR, PROCESSED_DATASET))

data = pd.read_pickle(path.join(PROCESSED_DIR, PROCESSED_DATASET))
interest = du.get_interest_dataset(data)
comprehension = du.get_comprehension_dataset(data)
comprehension = du.get_balanced_dataset(comprehension, 'comprehension_binary', random_state=1)
comprehension = comprehension.sample(frac=1, random_state=1)


/home/ubuntu/Question-Classifier/dataset_utils.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interest['interest_binary'] = (interest['interest'] > 3).astype(int)
/home/ubuntu/Question-Classifier/dataset_utils.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comprehension['comprehension_binary'] = (comprehension['comprehension'] > 3).astype(int)


In [13]:
# K-fold split
features = du.get_features(comprehension)
labels = du.get_labels(comprehension, 'comprehension_binary')
folds = du.kfold_split(features, labels, num_folds=8)

# Alternate to k-fold split: manually split

train_set, dev_set, test_set = du.split_datasets(comprehension, .2, DEV_RATIO)
train_features, train_labels = du.get_features(train_set), du.get_labels(train_set, 'comprehension_binary')
dev_features, dev_labels = du.get_features(dev_set), du.get_labels(dev_set, 'comprehension_binary')
test_features, test_labels = du.get_features(test_set), du.get_labels(test_set, 'comprehension_binary')

In [14]:
du.get_balance(train_labels)

{0: 322, 1: 326, 'percent_pos': 0.49691358024691357}

In [15]:
keras.backend.clear_session()
model = create_model(MAX_LEN, NUM_SX_FEATURES)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-5,
    decay_steps=50,
    decay_rate=0.9,
    staircase=True
)

model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [16]:
hist = model.fit(train_features, train_labels, validation_split=.1,batch_size=3, epochs=3, shuffle=True)

Epoch 1/3
195/195 [==============================] - 186s 878ms/step - loss: 0.7910 - accuracy: 0.4563 - val_loss: 0.7321 - val_accuracy: 0.5231
Epoch 2/3
195/195 [==============================] - 168s 863ms/step - loss: 0.7494 - accuracy: 0.4975 - val_loss: 0.7316 - val_accuracy: 0.5231
Epoch 3/3
195/195 [==============================] - 168s 863ms/step - loss: 0.7693 - accuracy: 0.4682 - val_loss: 0.7312 - val_accuracy: 0.5231


In [17]:
model.evaluate(dev_features, dev_labels, batch_size=BATCH_SIZE)

216/216 [==============================] - 43s 188ms/step - loss: 0.7716 - accuracy: 0.4645


[0.7716456055641174, 0.4645061790943146]

In [18]:
#model.save('model_checkpoints/dev62pct')

In [18]:
out = model.predict(dev_features, batch_size=BATCH_SIZE)

In [19]:
pos = np.where(out >= .5)
neg = np.where(out < .5)
print(f'Positives: {pos[0].shape[0]}')
print(f'Negatives: {neg[0].shape[0]}')

Positives: 648
Negatives: 0


dev62pct  dev_ac_59pct
